# Modeling
In this notebook, we'll be modeling the data we've previously prepared. Out notebook will be laid out as follows:

1. Model Selection & Generation
2. Hyperparameter Optimization
3. Fine-Tuning (if needed)
4. Reporting Best Model(s) + Settings
5. Interpretation
6. Conclusion

Our eventual goal here is two-fold:

1. Accurately [and fairly] model the diabetes dataset
2. Interpret the results to find something worth recommending to those wanting to reduce risk of diabetes. This can be via LIME/SHAP (i.e. some interpretable model that approximates the neural network) or via analyzing a more simple model's structure (i.e. regression coefficients, random forest decision boundaries)

In [1]:
# Environment Setup
from utils.model import *
from utils.dataset import *

***
## Model Selection & Generation

In [7]:
# generate lookup for models
models = {
    # "tree": TreeClassifier(target="diabetes", path="../datasets/processed.parquet"),
    "ffnn": MLPClassifier(target="diabetes", path="../datasets/processed.parquet")
}

# manual search
models["ffnn"].set_hyperparams({
    "learning_rate": .001,
    "batch_size": 64,
    "num_hidden": 16,
    "hidden_size": 64,
    "num_epochs": 50
})

# train & test basic model
for mt, model in models.items():
    # attempt to load, else train and test
    # if not model.load_model():
    model.train_model(verbose=1)
    model.test_model()

<Train-Test Split Report>
Train: 512887 obs, 170945 no diabetes [0], 171126 pre-diabetes [1], 170816 diabetes [2]
Test: 128222 obs, 42758 no diabetes [0], 42577 pre-diabetes [1], 42887 diabetes [2]


100%|██████████| 8014/8014 [00:16<00:00, 473.49it/s]


Epoch 1/50, Loss: 0.9859840509627923


100%|██████████| 8014/8014 [00:17<00:00, 456.88it/s]


Epoch 2/50, Loss: 0.978113356993506


...

Epoch 48/50, Loss: 0.8329552982568681


100%|██████████| 8014/8014 [00:17<00:00, 461.74it/s]


Epoch 49/50, Loss: 0.8318900789053089


100%|██████████| 8014/8014 [00:17<00:00, 452.85it/s]


Epoch 50/50, Loss: 0.8256360343679514

<Test Report>
Precision: [no diabetes] 0.6871986127277478, [pre-diabetes] 0.7103934968064675, [diabetes] 0.6428519602389533
Recall: [no diabetes] 0.6765751438327331, [pre-diabetes] 0.7471169880451887, [diabetes] 0.6197682281343997
F1-Score: [no diabetes] 0.6818455011490189, [pre-diabetes] 0.7282925991643294, [diabetes] 0.6310990811311347
Support: [no diabetes] 42758, [pre-diabetes] 42577, [diabetes] 42887
Accuracy: 68.0999%


***
## Hyperparameter Optimization

In [ ]:
# optimize hyperparams
optimizer_results = {model_type: model.optimize_hyperparams(kfold=2) for model_type, model in models.items()}
print(optimizer_results)

Fitting 2 folds for each of 360 candidates, totalling 720 fits


[CV 1/2] END batch_size=32, hidden_size=32, input_size=21, learning_rate=1, num_epochs=10, num_hidden=1, output_size=3;, score=nan total time=   0.2s
[CV 1/2] END batch_size=32, hidden_size=32, input_size=21, learning_rate=1, num_epochs=10, num_hidden=3, output_size=3;, score=nan total time=   0.2s
[CV 2/2] END batch_size=32, hidden_size=32, input_size=21, learning_rate=1, num_epochs=10, num_hidden=1, output_size=3;, score=nan total time=   0.2s
[CV 2/2] END batch_size=32, hidden_size=32, input_size=21, learning_rate=1, num_epochs=10, num_hidden=2, output_size=3;, score=nan total time=   0.3s
[CV 2/2] END batch_size=32, hidden_size=32, input_size=21, learning_rate=1, num_epochs=25, num_hidden=1, output_size=3;, score=nan total time=   0.3s
[CV 1/2] END batch_size=32, hidden_size=32, input_size=21, learning_rate=1, num_epochs=10, num_hidden=2, output_size=3;, score=nan total time=   0.3s
[CV 1/2] END batch_size=32, hidden_size=32, input_size=21, learning_rate=1, num_epochs=25, num_hidde

KeyboardInterrupt: 

***
## Fine-Tuning + Other Adjustments

***
## Best Model Report

***
## Interpretation

***
## Conclusion